![Logo](art/NYAN.png "Title")
## NotYetAnotherNightshade practical demonstration
As part of the manuscript "*Application of variational graph encoders as an effective generalist algorithm in holistic computer-aided drug design*".

We thank the anonymous reviewer for suggesting this!

**This notebook will go through the following:**
1. Downloading data from the TDCommons dataset, and converting it into latent space
2. Training a simple model on the latent space
3. Saving and loading models
4. Potentiating molecules based off existing uploaded models

In [ ]:
# PyTDC, Matplotlib and Seaborn are not part of the original Conda environment
# As I wanted to keep it minimal
# You'll have to install it here.

!pip3 install PyTDC
!pip3 install matplotlib
!pip3 install seaborn

In [ ]:
# Import libraries and set environment variables

import os
import sys
import math
import warnings
import json
import csv
import random
import loading
import numpy as np
from tqdm import tqdm
import joblib

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import metrics

dir_path = os.path.dirname(os.path.realpath("__file__")) + "/"

from data import BondType

import logging

from tdc.single_pred import ADME

warnings.filterwarnings("ignore")

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from spektral.data import BatchLoader, Dataset, Graph
from spektral import transforms

from model import VAE, NyanEncoder, NyanDecoder, EpochCounter

import scipy as sp
import tensorflow as tf

logging.getLogger("pysmiles").setLevel(logging.CRITICAL)

strategy = tf.distribute.MirroredStrategy()

save = dir_path + "saves/ZINC-extmodel5hk-3M"
batch_size = 1

#### Part 1
Downloading data from the TDCommons dataset, and converting it into latent space.

(We usually call this latentisation)

In [ ]:
# Please make sure that you have your Conda environment previously instantiated!
# I will use the dataset BBB_Martins as an example
dataset_name = "BBB_Martins"
data = ADME(name=dataset_name)

# Please take note! I are using random split here instead of scaffold split.
# You can refer to Supplementary File 1 for scaffold split results.
# The reason why random split is done here is for fivefold cross-validation.
split = data.get_split(method = "random")

# Combine all the data together in this case since I'm doing CV
data = split["train"].values.tolist() + split["valid"].values.tolist() + split["test"].values.tolist()

In [ ]:
with strategy.scope():

    encoder = NyanEncoder(latent_dim=64, batched=True)
    decoder = NyanDecoder(fingerprint_bits=679, regression=1613)

    model = VAE(encoder, decoder)
    model.load_weights(save).expect_partial()

print("Generating latents using the save {}".format(save))

all_smiles = data

# Initialise dataset
graph_data = list()
passed = list()

print("Loading {} molecules".format(len(all_smiles)))

for smile in tqdm(all_smiles):

    if smile[1] == "":
        continue

    try:

        graph = loading.get_data(smile[1], only_biggest=True, unknown_atom_is_dummy=True)

        x, a, e = loading.convert(*graph, bonds=[BondType.SINGLE, BondType.DOUBLE, BondType.TRIPLE, BondType.AROMATIC, BondType.NOT_CONNECTED])

        graph = Graph(x=np.array(x), a=np.array(a), e=np.array(e), y=np.array(0))

        graph_data.append([graph, None])

        passed.append(smile)

    except Exception as error:

        print("Errored loading SMILES", smile)

class EvalDataset (Dataset):

    def __init__ (self, **kwargs):
        super().__init__(**kwargs)

    def read (self):
        return [x[0] for x in graph_data]

dataset = EvalDataset()
loader = BatchLoader(dataset, batch_size=batch_size, epochs=1, mask=True, shuffle=False, node_level=False)

predictions = encoder.predict(loader.load())
predictions = [[float(y) for y in x] for x in predictions]

all_data = list()

for i in range(len(passed)):

    current_smiles = passed[i]

    appendable = [current_smiles] + predictions[i]
    all_data.append(appendable)


In [ ]:
# Now we have latentised the variables, let's look at the first molecules

print(all_data[0])

#### Part 2
Training a model and performing cross-validation

In [ ]:
# Usually I shuffle the data again before cross-validation, just in case
# Don't want to introduce biases unknowingly.

random.Random(0).shuffle(all_data)

# Fivefold cross-validation, as per manuscript
cross_validation_runs = 5

results = {"AUROC": list(), "AUPRC": list(), "accuracy": list()}

for i in range(cross_validation_runs):

    bottom = math.ceil(i/cross_validation_runs * len(all_data))
    top = math.ceil((i + 1)/cross_validation_runs * len(all_data))
    
    training_set = all_data[:bottom] + all_data[top:]
    testing_set = all_data[bottom:top]


    all_labels = sorted(list(set([x[0][2] for x in all_data])))
    
    training_latent_space = np.array([[float(y) for y in x[1:]] for x in training_set])
    training_labels = [all_labels.index(x[0][2]) for x in training_set]

    # Cap the latent spaces to [-1e8, 1e8]
    
    training_latent_space[training_latent_space == -np.inf] = -1e8
    training_latent_space[training_latent_space == np.inf] = 1e8
    
    # This is the part where you decide what kind of model you want to use
    # In this case, I'll follow what we originally did in the manuscript for the BBB_Martins dataset
    # Which is to use ExtraTreesClassifiers
    # You can use other pipelines of interest, even autoML methods

    clf = make_pipeline(RobustScaler(quantile_range=(10, 90), unit_variance=True), ExtraTreesClassifier(n_estimators=512, max_features="log2", n_jobs=-1, random_state=0))
    clf.fit(training_latent_space, training_labels)

    testing_latent_space = np.array([[float(y) for y in x[1:]] for x in testing_set])
    testing_labels = [all_labels.index(x[0][2]) for x in testing_set]
    
    # Test the model
    y_score = clf.predict_proba(testing_latent_space)
    y_test = testing_labels

    # Convert y_test to one-hot
    def create_one_hot(size, index):

        vector = [0] * size
        vector[index] = 1

        return vector

    y_test = np.array([create_one_hot(len(all_labels), x) for x in y_test])
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(len(all_labels)):

        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    results["AUROC"].append(roc_auc[0])

    # Strictly not equivalent, but more pessimistic
    auprc = metrics.average_precision_score(y_test[:, 0], y_score[:, 0])
    results["AUPRC"].append(auprc)

    accuracy = metrics.accuracy_score(y_test[:, 0], np.round(y_score[:, 0]))
    results["accuracy"].append(accuracy)

# Fivefold CV results
print(results)

In [ ]:
# Print the means and standard deviations

auroc_mean = str(np.mean(results["AUROC"]))
auroc_std = str(np.std(results["AUROC"]))

auprc_mean = str(np.mean(results["AUPRC"]))
auprc_std = str(np.std(results["AUPRC"]))

accuracy_mean = str(np.mean(results["accuracy"]))
accuracy_std = str(np.std(results["accuracy"]))

print(f"Accuracy: {accuracy_mean} ± {accuracy_std}")
print(f"AUROC: {auroc_mean} ± {auroc_std}")
print(f"AUPRC: {auprc_mean} ± {auprc_std}")

#### Part 3
Saving a model to Joblib so that it can be retrieved again later

In [ ]:
# Save a model
# I'm going to take the last model from the CV and save it.
# Of course, in practicable situations, you may want to fine-tune the exact
# train and test set

os.makedirs("models", exist_ok=True)

joblib.dump(clf, "models/" + dataset_name + ".joblib")

In [ ]:
# Loading a model is as simple as instantiating the Joblib

clf = joblib.load("models/" + dataset_name + ".joblib")

#### Part 4
Potentiating molecules

In [ ]:
# I'm going to use model which we have previously trained

# Here we try to potentiate a molecule that can pass the blood brain barrier
# I have blanked out the solubiity part since the model file itself is 2 GB,
# but it is there for your reference in case you need to do regression model potentiations.

# I deliberately soften out the labels in the preferences also, this is especially important when
# you have many, many parameters you want to optimise at once.

model_directory = "models/"

preferences = [

  #{"model": "Solubility_AqSolDB", "type": "regression", "weight": 1, "cutoff": 4, "offset": -8, "further-optimisation": false},
  {"model": "BBB_Martins", "type": "classification", "weight": [1, 0], "cutoff": [-0.6, 0.6], "offset": [0, 0], "further-optimisation": [True, True]}
    
]

# In the paper we did 10,000 iterations
# Here I'm just going to do 1,000 so that this doesn't take forever to run
# for smaller computers

iterations = 1000
step_std = 0.01

# Metropolis-Hastings configuration

beta = 1
annealing_alpha = 0.99
step_alpha = 1

latent = [0] * 64
latent = [x + random.gauss(0, 0.5) for x in latent]

trajectory = list()

previous_score = float("-inf")

def cost_function (x, cutoff, offset=0, allow_further_opt=True):

    if cutoff == 0:
        raise Exception("Cutoff cannot be zero")

    intersect = math.pow(10, -1.5) * cutoff

    if cutoff > 0 and (x - offset) < intersect or cutoff < 0 and (x - offset) > intersect:
        return -(-cutoff/abs(cutoff) * x + cutoff/abs(cutoff) * offset + (math.pow(10, -1.5) * abs(cutoff)) + 1.5)

    elif allow_further_opt:
        return -min(1.5, -math.log10((x - offset) / cutoff))

    else:
        return -max(0, min(1.5, -math.log10((x - offset)/ cutoff)))

# Load all models into memory
for model_pref in preferences:

    model_name = model_pref["model"]
    type = model_pref["type"]

    model_pref["clf"] = joblib.load(model_directory + "/" + model_name + ".joblib")

annealing_temp = 1
step_factor = 1

for i in range(iterations):

    annealing_temp *= annealing_alpha
    step_factor *= step_alpha

    score = float()
    model_scores = dict()

    for model_pref in preferences:

        model_name = model_pref["model"]
        type = model_pref["type"]

        if type == "classification":

            result = model_pref["clf"].predict_proba([latent])[0]
            model_scores[model_name] = result.tolist()

            for j in range(len(result)):

                score += cost_function(result[j], model_pref["cutoff"][j], offset=model_pref["offset"][j], allow_further_opt=model_pref["further-optimisation"][j]) * model_pref["weight"][j]

        elif type == "regression":

            result = model_pref["clf"].predict([latent])[0]
            model_scores[model_name] = result.tolist()

            score += cost_function(result, model_pref["cutoff"], offset=model_pref["offset"], allow_further_opt=model_pref["further-optimisation"]) * model_pref["weight"]

    status = None

    if score > previous_score:

        previous_latent = latent
        previous_score = score

        status = "Accepted"

        trajectory.append({"iteration": i + 1, "score": score, "latent": latent, "model_scores": model_scores})

    else:

        delta = score - previous_score
        accept_probability = math.exp(beta * delta / annealing_temp)

        if random.random() < accept_probability:

            previous_latent = latent
            previous_score = score

            status = "Accepted"

            trajectory.append({"iteration": i + 1, "score": score, "latent": latent, "model_scores": model_scores})

        else:

            latent = previous_latent

            status = "Declined"

    # Randomly move the points
    latent = [x + random.gauss(0, step_std) for x in latent]

    print(i, score, status, "TEMPERATURE:", annealing_temp)


In [ ]:
# Let's look at the results

print("\nFinal results")
print("===============")

for model_pref in preferences:

    model_name = model_pref["model"]
    type = model_pref["type"]

    if type == "classification":

        result = model_pref["clf"].predict_proba([latent])[0]
        print(model_name, result)

    elif type == "regression":

        result = model_pref["clf"].predict([latent])[0]
        print(model_name, result)

print("===============")

In [ ]:
# Typically potentiation works better on SVM models instead of ExtraTrees or other ensembling methods
# This is because those create a nice continuous probability distribution
# You can do a trade-off between the accuracy by changing the ExtraTreesClassifier in the earlier code segment to an SVM.
# Or you can do some tuning with the hyperparameters to get the MCMC to work better, or just increase the number
# of iterations.

# Anyway, let us now look into the latent space of the final molecule that was accepted in the MCMC

print(trajectory[-1])

# You can decode this latent space by turning it back into fingerprints, or what I usually do is do a 
# search within a library of ZINC molecule latent spaces;
# to see which one matches the best through Euclidean distance

In [ ]:
# Let us plot the trajectory of the entire potentiation, by doing a PCoA

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as col

from sklearn.decomposition import PCA

colours = ["#bdc5c8", "#75818d", "#364354", "#0a0c10"]
cmap = col.LinearSegmentedColormap.from_list("BW", colours, N=512)

sns.set_style("whitegrid", {"axes.grid": False, "figure.autolayout": True})

all_iterations = list()
all_latents = list()

for trajectory_iteration in trajectory:

    iteration = trajectory_iteration["iteration"]
    latent = trajectory_iteration["latent"]

    all_iterations.append(iteration)
    all_latents.append(latent)

pca = PCA(n_components=2)
pca.fit(all_latents)

transformed = pca.transform(all_latents).T

fig = plt.gcf()
fig.set_size_inches(6.5, 6.5)

ax = plt.gca()

for spine in ax.spines.values():
    spine.set_edgecolor("black")

ax.tick_params(reset=True, color="black", labelcolor="black", direction="out", which="both", length=5, width=1, top=False, right=False, labelsize=11)

points = ax.scatter(transformed[1], transformed[0], c=all_iterations, s=50, cmap=cmap)
fig.colorbar(points)

plt.ylabel("PC1 [{}%]".format(round(pca.explained_variance_ratio_[0] * 100, 3)), fontsize=13)
plt.xlabel("PC2 [{}%]".format(round(pca.explained_variance_ratio_[1] * 100, 3)), fontsize=13)

plt.show()

In [ ]:
# Let us also plot the trajectory of the BBB permeability

sns.set_style("whitegrid", {"axes.grid": True, "figure.autolayout": True})

fig = plt.gcf()

ax = plt.gca()

for spine in ax.spines.values():
    spine.set_edgecolor("black")

ax.tick_params(reset=True, color="black", labelcolor="black", direction="out", which="both", length=5, width=1, top=False, right=False, labelsize=13)

x = list()
y = list()

for iteration in trajectory:

    x.append(iteration["iteration"])
    y.append(iteration["model_scores"]["BBB_Martins"][1])

plt

plt.plot(x, y, color="#364354")
plt.show()

## That's it!

If you have any questions, please refer to the README for the contact details.

Obligatory cat pic.

![Logo](https://www.rd.com/wp-content/uploads/2021/01/GettyImages-1175550351.jpg "Katze")
(Credit: Getty Images)